# Machine Learning - Image Classification

In [1]:
#importing
import os

import torch
import torch.nn as nn
import torch.nn.functional as fun


from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from torch import optim
from tqdm import tqdm

from utils import *

## Database importing

In [2]:
load_dotenv()
DAT_PATH = os.getenv("TRAIN_DATASET_PATH")

ANIMALS_DATAFRAME = load_dataset_info("../data/archive/raw-img")

In [3]:
train_df, test_df = train_test_split(ANIMALS_DATAFRAME.sample(frac=0.5, random_state=37), test_size=0.01, random_state=37)
train_df, val_df = train_test_split(train_df, test_size=0.0526, random_state=37)

In [4]:
#setting up dataloaders

TARGET_SIZE = (256, 256)
MAX_SIZE = 500
BATCH_SIZE = 128
NUM_CLASSES = ANIMALS_DATAFRAME.label.nunique()
CLASS_LABELS = {name: idx for idx, name in enumerate(np.sort(ANIMALS_DATAFRAME.label.unique()))}

# vypocet frekvencie augmentacie -> aby nebol model biasnuty iba na majoritne categorie obrazkov
counts = train_df.label.value_counts()
max_count = counts.max()
aug_strength = (max_count / counts).to_dict()

print(aug_strength)
#
#   POMALE
#
# train_gen = AnimalImageGenerator(
#     df=train_df,
#     batch_size=BATCH_SIZE,
#     target_size=TARGET_SIZE,
#     num_classes=NUM_CLASSES,
#     augment=True,
#     shuffle=True,
#     aug_strength=aug_strength,
#     max_size=MAX_SIZE,
#     class_mapping=CLASS_LABELS,
# )
#
# test_gen = AnimalImageGenerator(
#     df=test_df,
#     batch_size=BATCH_SIZE,
#     target_size=TARGET_SIZE,
#     num_classes=NUM_CLASSES,
#     augment=False,
#     shuffle=False,
#     max_size=MAX_SIZE,
#     class_mapping=CLASS_LABELS,
# )
#
# val_gen = AnimalImageGenerator(
#     df=val_df,
#     batch_size=BATCH_SIZE,
#     target_size=TARGET_SIZE,
#     num_classes=NUM_CLASSES,
#     augment=False,
#     shuffle=False,
#     max_size=MAX_SIZE,
#     class_mapping=CLASS_LABELS,
# )


{'cane': 1.0, 'ragno': 1.0594104308390022, 'gallina': 1.5934515688949522, 'cavallo': 1.9194741166803615, 'farfalla': 2.3524672708962737, 'mucca': 2.636568848758465, 'scoiattolo': 2.6485260770975056, 'pecora': 2.7257876312718787, 'gatto': 3.0817941952506596, 'elefante': 3.4505169867060563}


In [5]:
from torch.utils.data import WeightedRandomSampler


#CHATOVINA ale ze vraj rychla

def create_sampler(df, class_mapping):
    labels = df["label"].map(class_mapping).values
    class_counts = np.bincount(labels)
    class_weights = 1.0 / class_counts

    sample_weights = class_weights[labels]
    sampler = WeightedRandomSampler(
        torch.from_numpy(sample_weights).float(),
        num_samples=len(sample_weights),
        replacement=True
    )
    return sampler

train_dataset = AnimalDataset(
    df=train_df,
    class_mapping=CLASS_LABELS,
    augment=True,
    target_size=(224, 224)
)

val_dataset = AnimalDataset(
    df=val_df,
    class_mapping=CLASS_LABELS,
    augment=False,
    target_size=(224, 224)
)

sampler = create_sampler(train_df, CLASS_LABELS)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    sampler=sampler,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)


In [10]:
#testovaci vypis

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)
print("Image dtype:", images.dtype)
print("Label sample:", labels[0])

Images shape: torch.Size([3, 224, 224])


AttributeError: 'int' object has no attribute 'shape'

## Model implemetation

In [12]:
#PRE TUTO FUNKCIU NEGENERUJ ZIADNE KOMENTARE
class ImageClassifier(nn.Module):
    def __init__(self, classes: int):
        super(ImageClassifier, self).__init__()
        self.numberOfClasses = classes

        # Convolution layres ONLY WORKS with RGB because of in_channels, kernel_size for filtering is 3 stride 1 padding 1 for size preservation
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1,padding=1)
        self.bn1 = nn.BatchNorm2d(16)

        # Significant for grad-CAM
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1,padding=1)
        self.bn2 = nn.BatchNorm2d(32)

        #
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1,padding=1)
        self.bn3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1,padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1,padding=1)
        self.bn5 = nn.BatchNorm2d(256)
        # Adaptive pooling to make model input-size agnostic / dont want to use it for now
        # self.adaptive_pool = nn.AdaptiveAvgPool2d((4, 4))

        #size is determined by conv channels and the reduction in size by conv channels
        #channels * width * height because 256 /2 /2 /2 /2 /2 is 8
        self.fc1 = nn.Linear(in_features=256*8*8, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=self.numberOfClasses)

    def forward(self, x):
        #Block 1
        x = fun.relu(self.bn1(self.conv1(x)))
        x = fun.max_pool2d(x, kernel_size=2) # zmensovanie velkosti

        x = fun.relu(self.bn2(self.conv2(x)))
        x = fun.max_pool2d(x, kernel_size=2)

        x = fun.relu(self.bn3(self.conv3(x)))
        x = fun.max_pool2d(x, kernel_size=2)

        x = fun.relu(self.bn4(self.conv4(x)))
        x = fun.max_pool2d(x, kernel_size=2)

        x = fun.relu(self.bn5(self.conv5(x)))
        x = fun.max_pool2d(x, kernel_size=2)

        x = x.reshape(x.size(0), -1)

        x = fun.relu(self.fc1(x))
        x = self.fc2(x)
        return x


### Grad CAM Implementation
 -- On hold
 https://medium.com/@codetrade/grad-cam-in-pytorch-a-powerful-tool-for-visualize-explanations-from-deep-networks-bdc7caf0b282



## Model Training

### Training function

In [13]:
def train_model(model: nn.Module,
                train_loader: AnimalImageGenerator,
                val_loader: AnimalImageGenerator,
                criterion: nn.Module,
                optimizer: torch.optim.Optimizer,
                device: torch.device,
                epochs: int = 10,
                scheduler=None):

    model.to(device)

    for epoch in range(1, epochs+1):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader, desc="Training"):
            images = torch.from_numpy(images).permute(0,3,1,2).float().to(device) # permute because the convolution layer gets input in the wrong order
            # images = torch.tensor(images, dtype=torch.float32, device=device)
            labels = torch.tensor(labels, dtype=torch.long, device=device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images = torch.tensor(images, dtype=torch.float32, device=device)
                labels = torch.tensor(labels, dtype=torch.long, device=device)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total

        if scheduler:
            scheduler.step(val_loss)

        print(f"Epoch {epoch}/{epochs} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
        ## TODO: add f1 score

### Training

In [14]:
img_class_model = ImageClassifier(NUM_CLASSES)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(img_class_model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

train_model(model=img_class_model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            device=device,
            epochs=1,
            scheduler=None)


device: cuda


Training:   0%|          | 0/192 [00:43<?, ?it/s]


KeyboardInterrupt: 

### Hyper parameter sweeping